In [1]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# importing the dataset

In [2]:
#import dataset
ds=pd.read_csv('/kaggle/input/amazon-product-reviews/Reviews.csv')

In [3]:
ds.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


# EDA

In [4]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568428 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [5]:
ds.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               26
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [6]:
ds=ds.dropna(how='any')

In [7]:
ds.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [8]:
#ds[ds['UserId']]

In [9]:
ds.drop(['ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Summary','Text'],axis=1,inplace=True)

In [10]:
ds['Time'].value_counts()

Time
1350345600    1143
1322179200    1088
1322438400    1070
1346889600    1018
1344211200     989
              ... 
1095638400       1
1102636800       1
1080259200       1
1087689600       1
1069718400       1
Name: count, Length: 3168, dtype: int64

In [11]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 568401 entries, 0 to 568453
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Id         568401 non-null  int64 
 1   ProductId  568401 non-null  object
 2   UserId     568401 non-null  object
 3   Score      568401 non-null  int64 
 4   Time       568401 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 26.0+ MB


# lets make more features:
    * day of the week
    * month
    *time of the day(Hour)

In [12]:
#change the time stamp into datatime
ds['Time']=pd.to_datetime(ds['Time'],unit='s',utc=True)
ds['Time']=ds['Time'].dt.tz_convert('America/New_York')

In [13]:
ds.head()

,Id,ProductId,UserId,Score,Time
0,1,B001E4KFG0,A3SGXH7AUHU8GW,5,2011-04-26 20:00:00-04:00
1,2,B00813GRG4,A1D87F6ZCVE5NK,1,2012-09-06 20:00:00-04:00
2,3,B000LQOCH0,ABXLMWJIXXAIN,4,2008-08-17 20:00:00-04:00
3,4,B000UA0QIQ,A395BORC6FGVXV,2,2011-06-12 20:00:00-04:00
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,5,2012-10-20 20:00:00-04:00


In [14]:
ds.head()

,Id,ProductId,UserId,Score,Time
0,1,B001E4KFG0,A3SGXH7AUHU8GW,5,2011-04-26 20:00:00-04:00
1,2,B00813GRG4,A1D87F6ZCVE5NK,1,2012-09-06 20:00:00-04:00
2,3,B000LQOCH0,ABXLMWJIXXAIN,4,2008-08-17 20:00:00-04:00
3,4,B000UA0QIQ,A395BORC6FGVXV,2,2011-06-12 20:00:00-04:00
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,5,2012-10-20 20:00:00-04:00


In [15]:
#year
ds['year']=ds['Time'].dt.year
#month
ds['month']=ds['Time'].dt.month
#week
ds['day_of_week']=ds['Time'].dt.dayofweek
#hour
ds['hour']=ds['Time'].dt.time

In [16]:
ds.head()

,Id,ProductId,UserId,Score,Time,year,month,day_of_week,hour
0,1,B001E4KFG0,A3SGXH7AUHU8GW,5,2011-04-26 20:00:00-04:00,2011,4,1,20:00:00
1,2,B00813GRG4,A1D87F6ZCVE5NK,1,2012-09-06 20:00:00-04:00,2012,9,3,20:00:00
2,3,B000LQOCH0,ABXLMWJIXXAIN,4,2008-08-17 20:00:00-04:00,2008,8,6,20:00:00
3,4,B000UA0QIQ,A395BORC6FGVXV,2,2011-06-12 20:00:00-04:00,2011,6,6,20:00:00
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,5,2012-10-20 20:00:00-04:00,2012,10,5,20:00:00


In [17]:
ds.isnull().sum()

Id             0
ProductId      0
UserId         0
Score          0
Time           0
year           0
month          0
day_of_week    0
hour           0
dtype: int64

In [18]:
#droping the time column
clean_data=ds.drop(['Time'],axis=1)

In [19]:
#dropign the ID
clean_data.drop(['Id'],axis=1,inplace=True)

In [20]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 568401 entries, 0 to 568453
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ProductId    568401 non-null  object
 1   UserId       568401 non-null  object
 2   Score        568401 non-null  int64 
 3   year         568401 non-null  int32 
 4   month        568401 non-null  int32 
 5   day_of_week  568401 non-null  int32 
 6   hour         568401 non-null  object
dtypes: int32(3), int64(1), object(3)
memory usage: 28.2+ MB


In [21]:
#convert the hour feature into int
clean_data['hour']=clean_data['hour'].apply(lambda x:x.hour )

# lets add another feature:
    * Name of the feature is: recomended
        *the feature if gonna be boolean
        * we are gonna assign it True if its equal or greater than 3 : False other wise

In [22]:
clean_data['Recomended']=clean_data['Score']>=3

In [23]:
clean_data.columns

Index(['ProductId', 'UserId', 'Score', 'year', 'month', 'day_of_week', 'hour',
       'Recomended'],
      dtype='object')

# split into train and test

In [24]:
#split into train and test
train,test=train_test_split(
    clean_data,
    random_state=42,
    test_size=0.2,
    stratify=clean_data['Recomended']
)

# turn them into torch dataset

In [25]:
class Mydataset(Dataset):
    def __init__(self,data):
        super().__init__()
        self.data=data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        #element to extract
        item=self.data.iloc[idx]
        
        #extracting features from the dataset
        productid=item['ProductId']
        userid=item['UserId']
        rate=item['Score']
        year=item['year']
        month=item['month']
        day_of_week=item['day_of_week']
        hour=item['hour']
        recomended=item['Recomended']
        
        return productid,userid,rate,year,month,day_of_week,hour,recomended

## loading the torch dataset format

In [26]:
#trainig dataset
train_dataset=Mydataset(train)
test_dataset=Mydataset(test)

In [27]:
print(train_dataset)
print(test_dataset)

# turning it into torch dataloader

In [28]:
#making the train dataloader
train_dataloader=DataLoader(
    dataset=train_dataset,
    batch_size=32,
    num_workers=os.cpu_count()
)

#making test dataloader
test_dataloader=DataLoader(
    dataset=test_dataset,
    batch_size=32,
    num_workers=os.cpu_count()
)